# Seq2Seq training

*kurziva* **tucne** ***underline***

## Data loading

#Take data from the file to proccess it.

In [1]:
import json 
filename = '/home/ksenia/projects/chatbot-messenger/conv_preprocess/outputs/conversations.json'

file = open(filename, "r")
if file.closed:
	sys.stderr.write("Couldn\'t open a file. \n")
	exit(-1)
file_data = json.load(file)
file.close()



## CoreNLP processing

In [2]:
import os
from stanfordcorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP(r'/home/ksenia/projects/chatbot-messenger/stanford-corenlp-full-2018-02-27/')


#Example how to use this library
#sentence = 'Hi Ksenia, how are you.'

#print('Tokenize:', nlp.word_tokenize(sentence))
#print('Part of Speech:', nlp.pos_tag(sentence))
#print('Constituency Parsing:', nlp.parse(sentence))
#print('Dependency Parsing:', nlp.dependency_parse(sentence))
#print('Named Entities:', nlp.ner(sentence))


it = len(file_data) #length of data 
i = 0
count = 0 #id for dict
dict_parse_sentence = {}
while i < it:
    iteration = len(file_data[i]["conversation"]) #iterator (count of messages in the dialog)
    k = 0
    while k < iteration:
        text_for_saving = ""
        raw_text = file_data[i]["conversation"][k]["raw_text"]
  
        #Delete <pause> from the text
        l = 0
        while l < len(raw_text):
            if raw_text[l] == "<":
                if raw_text[l:l+7] == "<pause>":
                    raw_text = raw_text[:l] + '&' + raw_text[l+7:]
            l += 1    
                
        result = nlp.ner(raw_text) #is a parsed sentence 
        counter = 0
        len_of_sentence = len(result)
        sentence_with_PH = ""     #sentence with PLACEHOLDERS
        name_PH_dict = {}
  
        #sentence with PLACEHOLDER
        while counter < len_of_sentence:
            word = result[counter][0]
            tag_entity = result[counter][1]
            if word == '&':
                sentence_with_PH += "<pause>"
            else:
                if tag_entity != 'O':
                    if tag_entity not in name_PH_dict:
                        name_PH_dict[tag_entity] = []
                    sentence_with_PH += '['
                    sentence_with_PH += tag_entity
                    sentence_with_PH += ']'
                    name_PH_dict[tag_entity].append(word) 
                else:
                    sentence_with_PH += word
            sentence_with_PH += " "       
            counter += 1
        #print("sentence: ", sentence_with_PH)
        
        if count not in dict_parse_sentence:
            dict_parse_sentence[count] = []
        dict_parse_sentence[count].append(sentence_with_PH)
        dict_parse_sentence[count].append(name_PH_dict)
            
        #print("RESULT: ", dict_parse_sentence)
        count += 1
        k += 1



nlp.close() # Do not forget to close! The backend server will consume a lot memery.

KeyboardInterrupt: 

## Save data to the file

In [3]:
len_dict = len(dict_parse_sentence)
it = len(file_data)
i = 0
print("LEN: ", len_dict)
counter = 0
while i < it:
    iteration = len(file_data[i]["conversation"]) #iterator (count of messages in the dialog)
    k = 0
    while k < iteration:
        file_data[i]["conversation"][k]["entities"] = []
        file_data[i]["conversation"][k]["named_entities"] = []
        if counter < len_dict:
            #Save to the structure the sentence with PH
            file_data[i]["conversation"][k]["entities"]  = dict_parse_sentence[counter][0]
            if (len(dict_parse_sentence[counter]) == 2):
                #Save to the structure named entities
                file_data[i]["conversation"][k]["named_entities"] = dict_parse_sentence[counter][1]
        counter += 1
        k += 1
    
    i += 1
print("FILEDATA: ", file_data)
#Please write the path to save file
path_for_saving = "/home/ksenia/data.txt"
file_for_saving = open(path_for_saving, "w")
file_for_saving.write(str(file_data))
file_for_saving.close()

LEN:  92
FILEDATA:  [{'worker_cnt': 55, 'conversation': [{'named_entities': {'PERSON': ['Betty']}, 'time': None, 'speaker': 'customer', 'raw_text': 'Hello? Is this Betty?', 'entities': 'Hello ? Is this [PERSON] ? '}, {'named_entities': {'PERSON': ['Alfred']}, 'time': None, 'speaker': 'worker', 'raw_text': "Yes, I'm here. Hi Alfred?", 'entities': "Yes , I 'm here . Hi [PERSON] ? "}, {'named_entities': {'DATE': ['tonight']}, 'time': None, 'speaker': 'customer', 'raw_text': 'Yes. How are you tonight?', 'entities': 'Yes . How are you [DATE] ? '}, {'named_entities': {}, 'time': None, 'speaker': 'worker', 'raw_text': "I'm ok.", 'entities': "I 'm ok . "}, {'named_entities': {}, 'time': None, 'speaker': 'customer', 'raw_text': "I'm glad that you accepted my chat invitation.", 'entities': "I 'm glad that you accepted my chat invitation . "}, {'named_entities': {}, 'time': None, 'speaker': 'worker', 'raw_text': "You're welcome.", 'entities': "You 're welcome . "}, {'named_entities': {}, 'time': 

## Separate question and answer


In [4]:
it = len(file_data)
i = 0
question_dict = {}
answer_dict = {}
counter_d = 0 #id for question and answer dictionary
while i < it:
    iteration = len(file_data[i]["conversation"])
    k = 0
    while k+1 < iteration:
        question_dict[counter_d] = file_data[i]["conversation"][k]["entities"]
        answer_dict[counter_d] = file_data[i]["conversation"][k+1]["entities"]
               
        counter_d += 1
        k += 2
    i += 1
        
 
#print("OK", question_dict)
#print("OK1", answer_dict)   

In [5]:
#def clean_text(text):
#    '''Clean text by removing unnecessary characters and altering the format of words.'''

#    text = text.lower()
    
#    text = re.sub(r"i'm", "i am", text)
#    text = re.sub(r"he's", "he is", text)
#    text = re.sub(r"she's", "she is", text)
#    text = re.sub(r"it's", "it is", text)
#    text = re.sub(r"that's", "that is", text)
#    text = re.sub(r"what's", "that is", text)
#    text = re.sub(r"where's", "where is", text)
#    text = re.sub(r"how's", "how is", text)
#    text = re.sub(r"\'ll", " will", text)
#    text = re.sub(r"\'ve", " have", text)
#    text = re.sub(r"\'re", " are", text)
#    text = re.sub(r"\'d", " would", text)
#    text = re.sub(r"\'re", " are", text)
#    text = re.sub(r"won't", "will not", text)
#    text = re.sub(r"can't", "cannot", text)
#    text = re.sub(r"n't", " not", text)
#    text = re.sub(r"n'", "ng", text)
#    text = re.sub(r"'bout", "about", text)
#    text = re.sub(r"'til", "until", text)
#    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
#    return text

## Sequence to sequence


In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import time
tf.__version__


'1.0.0'

In [7]:
#Lengths of sentences
lengths = []
for length in question_dict.values():
    lengths.append(len(length))
for length in answer_dict.values():
    lengths.append(len(length))

lengths = pd.DataFrame(lengths, columns=['counts'])

lengths.describe()

,counts
count,2362.000000
mean,2.483489
std,16.871087
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,303.000000


In [8]:
print(np.percentile(lengths, 80))
print(np.percentile(lengths, 85))
print(np.percentile(lengths, 90))
print(np.percentile(lengths, 95))
print(np.percentile(lengths, 99))

0.0
0.0
0.0
0.0
79.16999999999962


In [9]:
# Remove questions and answers that are shorter than 2 words and longer than 20 words.
#min_line_length = 2
max_line_length = 20

# Filter out the questions that are too short/long
short_questions_temp = []
short_answers_temp = []

i = 0
for question in question_dict.values():
    if len(str(question).split()) <= max_line_length:
        short_questions_temp.append(question_dict[i])
        short_answers_temp.append(answer_dict[i])
    i += 1

# Filter out the answers that are too short/long
short_questions = []
short_answers = []
i = 0
for answer in short_answers_temp:
    if len(str(answer).split()) <= max_line_length:
        short_answers.append(answer)
        short_questions.append(short_questions_temp[i])
    i += 1
        

In [10]:
# Compare the number of lines we will use with the total number of lines.
print("# of questions:", len(short_questions))
print("# of answers:", len(short_answers))
print("% of data used: {}%".format(round(len(short_questions)/len(question_dict),4)*100))



# of questions: 1166
# of answers: 1166
% of data used: 98.72999999999999%


In [11]:
# Create a dictionary for the frequency of the vocabulary
vocab = {}
for question in short_questions:
    for word in str(question).split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1
            
for answer in short_answers:
    for word in str(answer).split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

In [12]:
# Remove rare words from the vocabulary.
# We will aim to replace fewer than 5% of words with <UNK>
# You will see this ratio soon.
threshold = 10
count = 0
for k,v in vocab.items():
    if v >= threshold:
        count += 1

In [13]:
print("Size of total vocab:", len(vocab))
print("Size of vocab we will use:", count)





Size of total vocab: 215
Size of vocab we will use: 12


In [14]:
# In case we want to use a different vocabulary sizes for the source and target text, 
# we can set different threshold values.
# Nonetheless, we will create dictionaries to provide a unique integer for each word.
questions_vocab_to_int = {}

word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        questions_vocab_to_int[word] = word_num
        word_num += 1
        
answers_vocab_to_int = {}

word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        answers_vocab_to_int[word] = word_num
        word_num += 1



In [15]:
# Add the unique tokens to the vocabulary dictionaries.
codes = ['<PAD>','<EOS>','<UNK>','<GO>']

for code in codes:
    questions_vocab_to_int[code] = len(questions_vocab_to_int)+1
    
for code in codes:
    answers_vocab_to_int[code] = len(answers_vocab_to_int)+1

In [16]:
# Create dictionaries to map the unique integers to their respective words.
# i.e. an inverse dictionary for vocab_to_int.
questions_int_to_vocab = {v_i: v for v, v_i in questions_vocab_to_int.items()}
answers_int_to_vocab = {v_i: v for v, v_i in answers_vocab_to_int.items()}

In [17]:
# Check the length of the dictionaries.
print(len(questions_vocab_to_int))
print(len(questions_int_to_vocab))
print(len(answers_vocab_to_int))
print(len(answers_int_to_vocab))

16
16
16
16


In [18]:
# Add the end of sentence token to the end of every answer.
for i in range(len(short_answers)):
    short_answers[i] += ' <EOS>'



In [19]:
# Convert the text to integers. 
# Replace any words that are not in the respective vocabulary with <UNK> 
questions_int = []
for question in short_questions:
    ints = []
    for word in str(question).split():
        if word not in questions_vocab_to_int:
            ints.append(questions_vocab_to_int['<UNK>'])
        else:
            ints.append(questions_vocab_to_int[word])
    questions_int.append(ints)
    
answers_int = []
for answer in short_answers:
    ints = []
    for word in str(answer).split():
        if word not in answers_vocab_to_int:
            ints.append(answers_vocab_to_int['<UNK>'])
        else:
            ints.append(answers_vocab_to_int[word])
    answers_int.append(ints)



In [20]:
# Check the lengths
print(len(questions_int))
print(len(answers_int))



1166
1166


In [21]:
# Calculate what percentage of all words have been replaced with <UNK>
word_count = 0
unk_count = 0

for question in questions_int:
    for word in question:
        if word == questions_vocab_to_int["<UNK>"]:
            unk_count += 1
        word_count += 1
    
for answer in answers_int:
    for word in answer:
        if word == answers_vocab_to_int["<UNK>"]:
            unk_count += 1
        word_count += 1
    
unk_ratio = round(unk_count/word_count,4)*100
    
print("Total number of words:", word_count)
print("Number of times <UNK> is used:", unk_count)
print("Percent of words that are <UNK>: {}%".format(round(unk_ratio,3)))



Total number of words: 9730
Number of times <UNK> is used: 8336
Percent of words that are <UNK>: 85.67%


In [22]:
# Sort questions and answers by the length of questions.
# This will reduce the amount of padding during training
# Which should speed up training and help to reduce the loss

sorted_questions = []
sorted_answers = []

for length in range(1, max_line_length+1):
    for i in enumerate(questions_int):
        if len(i[1]) == length:
            sorted_questions.append(questions_int[i[0]])
            sorted_answers.append(answers_int[i[0]])

print(len(sorted_questions))
print(len(sorted_answers))
print()
for i in range(3):
    print(sorted_questions[i])
    print(sorted_answers[i])
    print()



1166
1166

[3]
[15, 15, 15, 15, 15, 15, 15]

[3]
[15, 15, 15, 15, 15, 15, 15]

[3]
[15, 15, 15, 15, 15, 15, 15]

[3]
[15, 15, 15, 15, 15, 15, 15]

[3]
[15, 15, 15, 15, 15, 15, 15]

[3]
[15, 15, 15, 15, 15, 15, 15]

[3]
[15, 15, 15, 15, 15, 15, 15]

[3]
[15, 15, 15, 15, 15, 15, 15]

[3]
[15, 15, 15, 15, 15, 15, 15]

[3]
[15, 15, 15, 15, 15, 15, 15]



In [23]:
def model_inputs():
    '''Create palceholders for inputs to the model'''
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    return input_data, targets, lr, keep_prob



In [24]:
def process_encoding_input(target_data, vocab_to_int, batch_size):
    '''Remove the last word id from each batch and concat the <GO> to the begining of each batch'''
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input


In [25]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
    '''Create the encoding layer'''
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    enc_cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    _, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = enc_cell,
                                                   cell_bw = enc_cell,
                                                   sequence_length = sequence_length,
                                                   inputs = rnn_inputs, 
                                                   dtype=tf.float32)
    return enc_state



In [26]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope,
                         output_fn, keep_prob, batch_size):
    '''Decode the training data'''
    
    attention_states = tf.zeros([batch_size, 1, dec_cell.output_size])
    
    att_keys, att_vals, att_score_fn, att_construct_fn = \
            tf.contrib.seq2seq.prepare_attention(attention_states,
                                                 attention_option="bahdanau",
                                                 num_units=dec_cell.output_size)
    
    train_decoder_fn = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0],
                                                                     att_keys,
                                                                     att_vals,
                                                                     att_score_fn,
                                                                     att_construct_fn,
                                                                     name = "attn_dec_train")
    train_pred, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, 
                                                              train_decoder_fn, 
                                                              dec_embed_input, 
                                                              sequence_length, 
                                                              scope=decoding_scope)
    train_pred_drop = tf.nn.dropout(train_pred, keep_prob)
    return output_fn(train_pred_drop)



In [27]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id,
                         maximum_length, vocab_size, decoding_scope, output_fn, keep_prob, batch_size):
    '''Decode the prediction data'''
    
    attention_states = tf.zeros([batch_size, 1, dec_cell.output_size])
    
    att_keys, att_vals, att_score_fn, att_construct_fn = \
            tf.contrib.seq2seq.prepare_attention(attention_states,
                                                 attention_option="bahdanau",
                                                 num_units=dec_cell.output_size)
    
    infer_decoder_fn = tf.contrib.seq2seq.attention_decoder_fn_inference(output_fn, 
                                                                         encoder_state[0], 
                                                                         att_keys, 
                                                                         att_vals, 
                                                                         att_score_fn, 
                                                                         att_construct_fn, 
                                                                         dec_embeddings,
                                                                         start_of_sequence_id, 
                                                                         end_of_sequence_id, 
                                                                         maximum_length, 
                                                                         vocab_size, 
                                                                         name = "attn_dec_inf")
    infer_logits, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, 
                                                                infer_decoder_fn, 
                                                                scope=decoding_scope)
    
    return infer_logits



In [28]:
def decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size,
                   num_layers, vocab_to_int, keep_prob, batch_size):
    '''Create the decoding cell and input the parameters for the training and inference decoding layers'''
    
    with tf.variable_scope("decoding") as decoding_scope:
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
        dec_cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
        
        weights = tf.truncated_normal_initializer(stddev=0.1)
        biases = tf.zeros_initializer()
        output_fn = lambda x: tf.contrib.layers.fully_connected(x, 
                                                                vocab_size, 
                                                                None, 
                                                                scope=decoding_scope,
                                                                weights_initializer = weights,
                                                                biases_initializer = biases)

        train_logits = decoding_layer_train(encoder_state, 
                                            dec_cell, 
                                            dec_embed_input, 
                                            sequence_length, 
                                            decoding_scope, 
                                            output_fn, 
                                            keep_prob, 
                                            batch_size)
        decoding_scope.reuse_variables()
        infer_logits = decoding_layer_infer(encoder_state, 
                                            dec_cell, 
                                            dec_embeddings, 
                                            vocab_to_int['<GO>'],
                                            vocab_to_int['<EOS>'], 
                                            sequence_length - 1, 
                                            vocab_size,
                                            decoding_scope, 
                                            output_fn, keep_prob, 
                                            batch_size)

    return train_logits, infer_logits



In [29]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size, sequence_length, answers_vocab_size, 
                  questions_vocab_size, enc_embedding_size, dec_embedding_size, rnn_size, num_layers, 
                  questions_vocab_to_int):
    
    '''Use the previous functions to create the training and inference logits'''
    
    enc_embed_input = tf.contrib.layers.embed_sequence(input_data, 
                                                       answers_vocab_size+1, 
                                                       enc_embedding_size,
                                                       initializer = tf.random_uniform_initializer(0,1))
    enc_state = encoding_layer(enc_embed_input, rnn_size, num_layers, keep_prob, sequence_length)

    dec_input = process_encoding_input(target_data, questions_vocab_to_int, batch_size)
    dec_embeddings = tf.Variable(tf.random_uniform([questions_vocab_size+1, dec_embedding_size], 0, 1))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    train_logits, infer_logits = decoding_layer(dec_embed_input, 
                                                dec_embeddings, 
                                                enc_state, 
                                                questions_vocab_size, 
                                                sequence_length, 
                                                rnn_size, 
                                                num_layers, 
                                                questions_vocab_to_int, 
                                                keep_prob, 
                                                batch_size)
    return train_logits, infer_logits



In [30]:
# Set the Hyperparameters
epochs = 100
batch_size = 128
rnn_size = 512
num_layers = 2
encoding_embedding_size = 512
decoding_embedding_size = 512
learning_rate = 0.005
learning_rate_decay = 0.9
min_learning_rate = 0.0001
keep_probability = 0.75



In [31]:
# Reset the graph to ensure that it is ready for training
tf.reset_default_graph()
# Start the session
sess = tf.InteractiveSession()

# Load the model inputs    
input_data, targets, lr, keep_prob = model_inputs()
# Sequence length will be the max line length for each batch
sequence_length = tf.placeholder_with_default(max_line_length, None, name='sequence_length')
# Find the shape of the input data for sequence_loss
input_shape = tf.shape(input_data)

# Create the training and inference logits
train_logits, inference_logits = seq2seq_model(
    tf.reverse(input_data, [-1]), targets, keep_prob, batch_size, sequence_length, len(answers_vocab_to_int), 
    len(questions_vocab_to_int), encoding_embedding_size, decoding_embedding_size, rnn_size, num_layers, 
    questions_vocab_to_int)



# Create a tensor for the inference logits, needed if loading a checkpoint version of the model
tf.identity(inference_logits, 'logits')

with tf.name_scope("optimization"):
    # Loss function
    cost = tf.contrib.seq2seq.sequence_loss(
        train_logits,
        targets,
        tf.ones([input_shape[0], sequence_length]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)


In [32]:
def pad_sentence_batch(sentence_batch, vocab_to_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]



In [33]:
def batch_data(questions, answers, batch_size):
    """Batch questions and answers together"""
    for batch_i in range(0, len(questions)//batch_size):
        start_i = batch_i * batch_size
        questions_batch = questions[start_i:start_i + batch_size]
        answers_batch = answers[start_i:start_i + batch_size]
        pad_questions_batch = np.array(pad_sentence_batch(questions_batch, questions_vocab_to_int))
        pad_answers_batch = np.array(pad_sentence_batch(answers_batch, answers_vocab_to_int))
        yield pad_questions_batch, pad_answers_batch

In [34]:
# Validate the training with 10% of the data
train_valid_split = int(len(sorted_questions)*0.15)

# Split the questions and answers into training and validating data
train_questions = sorted_questions[train_valid_split:]
train_answers = sorted_answers[train_valid_split:]

valid_questions = sorted_questions[:train_valid_split]
valid_answers = sorted_answers[:train_valid_split]

print(len(train_questions))
print(len(valid_questions))



992
174


In [35]:
display_step = 100 # Check training loss after every 100 batches
stop_early = 0 
stop = 5 # If the validation loss does decrease in 5 consecutive checks, stop training
validation_check = ((len(train_questions))//batch_size//2)-1 # Modulus for checking validation loss
total_train_loss = 0 # Record the training loss for each display step
summary_valid_loss = [] # Record the validation loss for saving improvements in the model

checkpoint = "best_model.ckpt" 

sess.run(tf.global_variables_initializer())

for epoch_i in range(1, epochs+1):
    for batch_i, (questions_batch, answers_batch) in enumerate(
            batch_data(train_questions, train_answers, batch_size)):
        start_time = time.time()
        _, loss = sess.run(
            [train_op, cost],
            {input_data: questions_batch,
             targets: answers_batch,
             lr: learning_rate,
             sequence_length: answers_batch.shape[1],
             keep_prob: keep_probability})

        total_train_loss += loss
        end_time = time.time()
        batch_time = end_time - start_time

        if batch_i % display_step == 0:
            print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                  .format(epoch_i,
                          epochs, 
                          batch_i, 
                          len(train_questions) // batch_size, 
                          total_train_loss / display_step, 
                          batch_time*display_step))
            total_train_loss = 0

        if batch_i % validation_check == 0 and batch_i > 0:
            total_valid_loss = 0
            start_time = time.time()
            for batch_ii, (questions_batch, answers_batch) in \
                    enumerate(batch_data(valid_questions, valid_answers, batch_size)):
                valid_loss = sess.run(
                cost, {input_data: questions_batch,
                       targets: answers_batch,
                       lr: learning_rate,
                       sequence_length: answers_batch.shape[1],
                       keep_prob: 1})
                total_valid_loss += valid_loss
            end_time = time.time()
            batch_time = end_time - start_time
            avg_valid_loss = total_valid_loss / (len(valid_questions) / batch_size)
            print('Valid Loss: {:>6.3f}, Seconds: {:>5.2f}'.format(avg_valid_loss, batch_time))
            
            # Reduce learning rate, but not below its minimum value
            learning_rate *= learning_rate_decay
            if learning_rate < min_learning_rate:
                learning_rate = min_learning_rate

            summary_valid_loss.append(avg_valid_loss)
            if avg_valid_loss <= min(summary_valid_loss):
                print('New Record!') 
                stop_early = 0
                saver = tf.train.Saver() 
                saver.save(sess, checkpoint)

            else:
                print("No Improvement.")
                stop_early += 1
                if stop_early == stop:
                    break
    
    if stop_early == stop:
        print("Stopping Training.")
        break



Epoch   1/100 Batch    0/7 - Loss:  0.029, Seconds: 38.89
Valid Loss:  0.000, Seconds:  0.12
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Epoch   2/100 Batch    0/7 - Loss:  0.000, Seconds: 30.18
Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Epoch   3/100 Batch    0/7 - Loss:  0.000, Seconds: 29.62
Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Epoch   4/100 Batch    0/7 - Loss:  0.000, Seconds: 29.84
Valid Loss:  0.000, Seconds:  0.10
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Epoch   5/100 Batch    0/7 - Loss:  0.000, Seconds: 29.95
Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Epoch

Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Epoch  43/100 Batch    0/7 - Loss:  0.000, Seconds: 30.22
Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Epoch  44/100 Batch    0/7 - Loss:  0.000, Seconds: 29.89
Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Epoch  45/100 Batch    0/7 - Loss:  0.000, Seconds: 30.17
Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Epoch  46/100 Batch    0/7 - Loss:  0.000, Seconds: 29.94
Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Epoch  47/100 Batch    0/7 - Loss:  0.000, Seconds: 29.70
Valid

Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Epoch  84/100 Batch    0/7 - Loss:  0.000, Seconds: 30.22
Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Epoch  85/100 Batch    0/7 - Loss:  0.000, Seconds: 30.52
Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Epoch  86/100 Batch    0/7 - Loss:  0.000, Seconds: 30.32
Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Epoch  87/100 Batch    0/7 - Loss:  0.000, Seconds: 30.21
Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid Loss:  0.000, Seconds:  0.09
New Record!
Epoch  88/100 Batch    0/7 - Loss:  0.000, Seconds: 30.34
Valid Loss:  0.000, Seconds:  0.09
New Record!
Valid

In [43]:

def question_to_seq(question, vocab_to_int):
    '''Prepare the question for the model'''
    
#    question = clean_text(question)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in str(question).split()]

In [44]:
# Create your own input question
#input_question = 'How are you?'

# Use a question from the data as your input
random = np.random.choice(len(short_questions))
input_question = short_questions[random]

# Prepare the question
input_question = question_to_seq(input_question, questions_vocab_to_int)

# Pad the questions until it equals the max_line_length
input_question = input_question + [questions_vocab_to_int["<PAD>"]] * (max_line_length - len(input_question))
# Add empty questions so the the input_data is the correct shape
batch_shell = np.zeros((batch_size, max_line_length))
# Set the first question to be out input question
batch_shell[0] = input_question    
    
# Run the model with the input question
answer_logits = sess.run(inference_logits, {input_data: batch_shell, 
                                            keep_prob: 1.0})[0]

# Remove the padding from the Question and Answer
pad_q = questions_vocab_to_int["<PAD>"]
pad_a = answers_vocab_to_int["<PAD>"]

print('Question')
print('  Word Ids:      {}'.format([i for i in input_question if i != pad_q]))
print('  Input Words: {}'.format([questions_int_to_vocab[i] for i in input_question if i != pad_q]))

print('\nAnswer')
print('  Word Ids:      {}'.format([i for i in np.argmax(answer_logits, 1) if i != pad_a]))
print('  Response Words: {}'.format([answers_int_to_vocab[i] for i in np.argmax(answer_logits, 1) if i != pad_a]))



Question
  Word Ids:      [8, 15, 10, 15, 7, 9, 15, 15, 15, 4, 15, 15, 4, 15, 5, 15, 15, 11]
  Input Words: ['I', '<UNK>', "n't", '<UNK>', 'that', 'this', '<UNK>', '<UNK>', '<UNK>', '.', '<UNK>', '<UNK>', '.', '<UNK>', 'you', '<UNK>', '<UNK>', '?']

Answer
  Word Ids:      [15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15]
  Response Words: ['<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>']
